# 공유

In [ ]:
# 크롤러 만들기

from selenium import webdriver
from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

import math
import time
import pandas as pd

In [ ]:
# datas = []
data = {}
url = 'https://www.kyobobook.co.kr/bestSellerNew/bestseller.laf?range=1&kind=3&orderClick=DAC&mallGb=KOR&linkClass=A'
driver = webdriver.Chrome('C:/test/chromedriver.exe')
driver.implicitly_wait(10)
driver.get(url)
driver.find_element_by_xpath('//*[@id="main_contents"]/div[2]/ul/li[11]/a').click()  # 경제경영 카테코리 클릭
book_title, book_category, book_hash_tag, book_introduce, book_publisher, book_year = [], [], [], [], [], []
# 연도 변경
for y_click in range(1, 11): # 2012년까지 하려면 11로 수정필요
    y_click = str(y_click)
    driver.find_element_by_xpath('//*[@id="main_contents"]/div[3]/div/span[2]/a').click() # 연도 더보기 클릭
    driver.find_element_by_xpath('//*[@id="main_contents"]/div[3]/div/span[2]/ul/li['+y_click+']/a').click()  # 연도 클릭 (for문 필요)
    year = driver.find_element_by_xpath('//*[@id="main_contents"]/div[3]/div/span[2]/a').text #연도 텍스트 추출
    print('='*10, year, '='*10)
    for page in range(2):
        parentelement = driver.find_element_by_class_name('list_type01')
        book_count = len(parentelement.find_elements_by_class_name('cover'))
        for book_list in range(1, book_count+1):  # 책 리스트개수이므로 21로 수정필요
            book_year.append(year)
            element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="main_contents"]/ul/li['+str(book_list)+']/div[2]/div[2]/a')))
            element.click()
            time.sleep(5)
            # 텍스트 크롤링 시작
            title = list(driver.find_element_by_xpath('//*[@id="container"]/div[2]/form/div[1]/h1/strong').text.splitlines())  # 책 제목 출력
            category = list(driver.find_element_by_xpath('//*[@id="container"]/div[5]/div[1]/div[*]/ul/li/a[1]').text.split(' '))  # 카테고리 출력
            hash_tag = list(driver.find_element_by_xpath('//*[@id="container"]/div[5]/div[1]/div[*]/div[2]').text.splitlines())  # 해시태그 출력
            introduce = driver.find_element_by_xpath('//*[@id="container"]/div[5]/div[1]/div[*]/div[4]').text  # 소개글 출력
            print('='*10, title[0]+' 수집 시작','='*10)
            book_title += title
            book_category += category
            book_hash_tag += hash_tag
            book_introduce.append(introduce)
            # 출판사 총평
            anchor_xpath='''//*[@id="container"]/div[5]/div[1]/div/h2[contains(text(), '출판사 서평')]//following-sibling::div/div/a'''
            driver.find_element_by_xpath(anchor_xpath).click()
            anchor_xpath = '''//*[@id="container"]/div[5]/div[1]/div/h2[contains(text(), '출판사 서평')]//following-sibling::div/div[2]'''
            publisher = driver.find_element_by_xpath(anchor_xpath).text
            book_publisher.append(publisher)
            print('='*10, '완료','='*10)
            time.sleep(3)
            driver.back()  # 뒤로가기 버튼
            time.sleep(5)
        try:
            driver.find_element_by_xpath('//*[@id="main_contents"]/div[4]/div[1]/ul/li[2]/a').click() # 2페이지로 이동
            time.sleep(3)
        except:
            pass
data["year"] = book_year
data["book_name"] = book_title
data["book_categories"] = book_category
data["hash"] = book_hash_tag
data["intro"] = book_introduce
data["pub"] = book_publisher
data